In [0]:
from google.colab import drive
import os

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ（半角英数のみで指定してください）
base_path = "/gdrive/My Drive/autotrace"

! echo "autotraceフォルダの中身 -----------"
! ls -l "$base_path"
! echo "--------------------"

In [0]:
# clone
! git clone  --depth 1 https://github.com/miu200521358/AlphaPose-vmd.git

In [0]:
! mkdir -p ./AlphaPose-vmd/detector/yolo/data

# yolov3-spp.weightsのダウンロード(YOLOモデル)
file_id = "1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC"
file_name = "yolov3-spp.weights"
! cd  ./AlphaPose-vmd/detector/yolo/data && curl -sc ./cookie "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
code = "$(awk '/_warning_/ {print $NF}' ./cookie)"  
! cd  ./AlphaPose-vmd/detector/yolo/data && curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$code&id=$file_id" -o "$file_name"

In [0]:
! mkdir -p ./AlphaPose-vmd/detector/tracker/data

# jde.uncertainty.ptのダウンロード(Trackerモデル)
file_id = "1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA"
file_name = "jde.1088x608.uncertainty.pt"
! cd  ./AlphaPose-vmd/detector/tracker/data && curl -sc ./cookie "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
code = "$(awk '/_warning_/ {print $NF}' ./cookie)"  
! cd  ./AlphaPose-vmd/detector/tracker/data && curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$code&id=$file_id" -o "$file_name"

In [0]:
! mkdir -p ./AlphaPose-vmd/pretrained_models

# 学習済みデータのダウンロード(Simple Baseline)
file_id = "1nxyfUbvWDaaT9eDM7Y31ScSVPlGy6gfw"
file_name = "simple_res50_256x192.pth"
! cd  ./AlphaPose-vmd/pretrained_models && curl -sc ./cookie "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
code = "$(awk '/_warning_/ {print $NF}' ./cookie)"  
! cd  ./AlphaPose-vmd/pretrained_models && curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$code&id=$file_id" -o "$file_name"

In [0]:
# 学習済みデータのダウンロード(Fast Pose)
file_id = "1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn"
file_name = "fast_res50_256x192.pth"
! cd  ./AlphaPose-vmd/pretrained_models && curl -sc ./cookie "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
code = "$(awk '/_warning_/ {print $NF}' ./cookie)"  
! cd  ./AlphaPose-vmd/pretrained_models && curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$code&id=$file_id" -o "$file_name"

In [0]:
# AlphaPose のライブラリインストール・ビルド
# https://github.com/MVIG-SJTU/AlphaPose/issues/492
! cd ./AlphaPose-vmd && python ./setup.py build develop

In [0]:
# ライブラリのアップデート

# YAML（最新版でないとFullLoaderが使えない）
# https://stackoverflow.com/questions/55551191/module-yaml-has-no-attribute-fullloader
pip install -U PyYAML

In [0]:
! cd AlphaPose-vmd && git pull

In [0]:
import datetime

# 処理日時
now_str = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# 入力動画ファイル（半角英数のみで指定してください）
input_name = "input"
input_video = "{0}/{1}.mp4".format(base_path, input_name)

# 出力先（サブディレクトリを生成します）
outdir_path = "{0}/output_{1}_{2}".format(base_path, input_name, now_str)
! mkdir "$outdir_path"

# Simple Baseline
cfg_path = "configs/coco/resnet/256x192_res50_lr1e-3_1x-simple.yaml"
ckp_path = "pretrained_models/simple_res50_256x192.pth"

# Fast Pose
#cfg_path = "configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml"
#ckp_path = "pretrained_models/fast_res50_256x192.pth"

# --format open: Openposeライクな出力形式
# --detector "tracker": 追跡ありの検出方法
# --detbatch 4: バッチサイズ
# --save_video: 解析結果を動画で保存する
! cd AlphaPose-vmd && python scripts/demo_inference.py --cfg "$cfg_path" --checkpoint "$ckp_path" --video "$input_video" --outdir "$outdir_path" --detector "tracker" --format "open" --detbatch 1 --save_video